# BERT (Bidirectional Encoder Representations from Transformers)

# Question-Answering

In [26]:
#!pip install transformers
#!pip install pytorch 

## PATH DIR

In [1]:
import os
import sys
PATH = os.getcwd()
DIR_DATA = PATH + '{0}data{0}'.format(os.sep)
sys.path.append(PATH) if PATH not in list(sys.path) else None
DIR_DATA

'C:\\Users\\epuerta\\OneDrive - Universidad Tecnológica de Bolívar\\Apps\\AppsISCO\\InteligenciaArtificial-ISCO-A06A\\laboratories\\data\\'

In [3]:
import nltk
# Descargar recursos necesarios para el tokenizador de oraciones de NLTK
# nltk.download('punkt')

In [4]:
filename = DIR_DATA + 'AboutMe.txt'
filename

'C:\\Users\\epuerta\\OneDrive - Universidad Tecnológica de Bolívar\\Apps\\AppsISCO\\InteligenciaArtificial-ISCO-A06A\\laboratories\\data\\AboutMe.txt'

In [5]:
# Abre el archivo en modo lectura
with open(filename, 'r') as archivo:
    contexto = archivo.read()

# Muestra el contenido del archivo
print(contexto)

Edwin Puertas is an AI Software Architect and NLP Researcher. Currently, I am an Associate Professor at the Universidad TecnolÃ³gica de BolÃ­var, and I simultaneously serve as a bridge between academia and industry in projects related to AI. My work focuses on developing innovative AI solutions that bridge the gap between theoretical research and practical applications. With over 19 years of experience in academia, I have taught courses in Artificial Intelligence, Natural Language Processing, Big Data, and Software Engineering, fostering an environment of collaborative learning and research excellence.

As an AI Software Architect, I lead projects involving the design and implementation of scalable AI systems, applying advanced machine learning techniques to solve complex problems across various industries. My research in NLP explores the intricacies of human language, aiming to enhance natural language understanding and develop applications that improve human-computer interaction.

I 

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
the_model = 'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
tokenizer = AutoTokenizer.from_pretrained(the_model, do_lower_case=False)
model = AutoModelForQuestionAnswering.from_pretrained(the_model)


Some weights of the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
print(model)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [12]:
pregunta = "¿Cuantos años de experiencia tiene?"

#Tokenizar el contexto por oraciones usando nltk
oraciones_contexto = nltk.sent_tokenize(contexto)

# Añadir la pregunta como la primera oración
oraciones = [pregunta] + oraciones_contexto

# Procesar cada oración por separado
for idx, oracion in enumerate(oraciones):
    # Codificar la oración sola, utilizando encode_plus para cada oración
    encode = tokenizer.encode_plus(oracion, return_tensors="pt", add_special_tokens=True)
    
    # Obtener los input_ids y convertirlos en tokens
    input_ids = encode["input_ids"]
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    
    # Mostrar la oración original y sus tokens
    print(f"Oración {idx + 1}:")
    print(f"Texto original: {oracion}")
    print(f"Tokens de BERT: {tokens}")
    print("-" * 50)  # Separador entre oraciones

Oración 1:
Texto original: ¿Cuantos años de experiencia tiene?
Tokens de BERT: ['[CLS]', '¿', 'Cuanto', '##s', 'años', 'de', 'experiencia', 'tiene', '?', '[SEP]']
--------------------------------------------------
Oración 2:
Texto original: Edwin Puertas is an AI Software Architect and NLP Researcher.
Tokens de BERT: ['[CLS]', '[UNK]', 'Puerta', '##s', 'is', 'an', 'AI', '[UNK]', 'Archi', '##tec', '##t', 'and', 'NL', '##P', 'Research', '##er', '.', '[SEP]']
--------------------------------------------------
Oración 3:
Texto original: Currently, I am an Associate Professor at the Universidad TecnolÃ³gica de BolÃ­var, and I simultaneously serve as a bridge between academia and industry in projects related to AI.
Tokens de BERT: ['[CLS]', 'Cur', '##ren', '##t', '##ly', ',', 'I', 'am', 'an', 'Associa', '##te', 'Profes', '##sor', 'at', 'the', 'Universidad', '[UNK]', 'de', '[UNK]', ',', 'and', 'I', 'simul', '##tan', '##eo', '##us', '##ly', 'serv', '##e', 'as', 'a', 'bri', '##dge', '[UNK]', 'a

In [13]:
# Ejemplo pregunta respuesta
nlp = pipeline("question-answering", model=model, tokenizer=tokenizer)
salida = nlp({"question": pregunta, "context":contexto})
print(salida)

{'score': 0.24410812556743622, 'start': 378, 'end': 385, 'answer': 'over 19'}


In [14]:
from textwrap import wrap

def pregunta_respuesta(model, contexto, nlp):

  # Imprimir contexto
  print('Contexto:')
  print('-----------------')
  print('\n'.join(wrap(contexto)))

  # Loop preguntas-respuestas:
  continuar = True
  while continuar:
    print('\nPregunta:')
    print('-----------------')
    pregunta = str(input())

    continuar = pregunta!=''

    if continuar:
      salida = nlp({'question':pregunta, 'context':contexto})
      print('\nRespuesta:')
      print('-----------------')
      print(salida['answer'])

In [15]:
pregunta_respuesta(model, contexto, nlp)

Contexto:
-----------------
Edwin Puertas is an AI Software Architect and NLP Researcher.
Currently, I am an Associate Professor at the Universidad TecnolÃ³gica
de BolÃ­var, and I simultaneously serve as a bridge between academia
and industry in projects related to AI. My work focuses on developing
innovative AI solutions that bridge the gap between theoretical
research and practical applications. With over 19 years of experience
in academia, I have taught courses in Artificial Intelligence, Natural
Language Processing, Big Data, and Software Engineering, fostering an
environment of collaborative learning and research excellence.  As an
AI Software Architect, I lead projects involving the design and
implementation of scalable AI systems, applying advanced machine
learning techniques to solve complex problems across various
industries. My research in NLP explores the intricacies of human
language, aiming to enhance natural language understanding and develop
applications that improve hum

 ¿donde estudio?



Respuesta:
-----------------
Universidad TecnolÃ³gica de BolÃ­var

Pregunta:
-----------------


KeyboardInterrupt: Interrupted by user

In [ ]:
contexto = "Avengers: Endgame (Vengadores: Endgame en España)2​ es una película de superhéroes estadounidense de 2019 basada en el equipo de superhéroes Los Vengadores de Marvel Comics. Producida por Marvel Studios y distribuida por Walt Disney Studios Motion Pictures, es la secuela directa de Avengers: Infinity War (2018) y la película número 22 del Universo cinematográfico de Marvel (UCM), y hasta la fecha la más larga. Dirigida por Anthony y Joe Russo y escrita por Christopher Markus y Stephen McFeely, la película cuenta con un reparto coral que incluye a Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, Jeremy Renner, Don Cheadle, Paul Rudd, Brie Larson, Karen Gillan, Danai Gurira, Benedict Wong, Jon Favreau, Bradley Cooper, Gwyneth Paltrow y Josh Brolin. En la película, los miembros supervivientes de Los Vengadores y sus aliados intentan revertir el daño causado por Thanos en Infinity War."
pregunta_respuesta(model, contexto, nlp)

Contexto:
-----------------
Avengers: Endgame (Vengadores: Endgame en España)2​ es una película de
superhéroes estadounidense de 2019 basada en el equipo de superhéroes
Los Vengadores de Marvel Comics. Producida por Marvel Studios y
distribuida por Walt Disney Studios Motion Pictures, es la secuela
directa de Avengers: Infinity War (2018) y la película número 22 del
Universo cinematográfico de Marvel (UCM), y hasta la fecha la más
larga. Dirigida por Anthony y Joe Russo y escrita por Christopher
Markus y Stephen McFeely, la película cuenta con un reparto coral que
incluye a Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris
Hemsworth, Scarlett Johansson, Jeremy Renner, Don Cheadle, Paul Rudd,
Brie Larson, Karen Gillan, Danai Gurira, Benedict Wong, Jon Favreau,
Bradley Cooper, Gwyneth Paltrow y Josh Brolin. En la película, los
miembros supervivientes de Los Vengadores y sus aliados intentan
revertir el daño causado por Thanos en Infinity War.

Pregunta:
-----------------


 ¿quien Chris evans?



Respuesta:
-----------------
de Marvel (UCM), y hasta la

Pregunta:
-----------------
